In [ ]:
# Professional Library Imports and Configurations

# Core Libraries for Data Manipulation and Computation
import pandas as pd  # Data manipulation and analysis
import numpy as np  # Numerical operations and matrix computations

# Advanced Data Visualization Libraries
import matplotlib.pyplot as plt  # Static plotting
import seaborn as sns  # Statistical data visualization
import plotly.express as px  # Interactive plotting
import plotly.graph_objects as go  # Advanced interactive visualizations

# Machine Learning and Evaluation Tools
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb  # High-performance gradient boosting library
# Utilities and System Tools
import os  # Operating system utilities
import time  # Time performance tracking
import warnings  # Warning suppression for clean output

# Suppress Non-Critical Warnings for Better Readability
warnings.filterwarnings("ignore")

# Configure Matplotlib Visualization Defaults
plt.style.use("ggplot")  # Set a professional and clean style
plt.rcParams.update({
    "figure.figsize": [12, 6],  # Default figure size
    "axes.labelsize": 14,  # Label font size
    "axes.titlesize": 16,  # Title font size
    "xtick.labelsize": 12,  # X-axis tick font size
    "ytick.labelsize": 12,  # Y-axis tick font size
    "legend.fontsize": 12,  # Legend font size
    "grid.color": "#d3d3d3",  # Grid color for better readability
    "grid.linestyle": "--"  # Dashed grid lines
})

# Configure Seaborn Visualization Defaults
sns.set_theme(
    style="whitegrid",  # White grid background for clarity
    rc={"axes.facecolor": "#f9f9f9"}  # Light grey axes background
)

# Configure Plotly Default Settings for Interactive Visualizations
px.defaults.template = "plotly_white"  # Minimalist white theme for clarity
px.defaults.width = 1000  # Standard width for plots
px.defaults.height = 600  # Standard height for plots
px.defaults.color_continuous_scale = px.colors.sequential.Viridis  # Aesthetic color scale

In [ ]:
file_path = '/content/BKB_WaterQualityData_2020084.csv'
data = pd.read_csv(file_path)
print(data.head()) #Side ID gives the degree of closure of site x100km
print(data.shape)

In [ ]:
print(data.isna().sum())
data = data.drop(['Unit_Id','Read_Date','Secchi Depth (m)','Air Temp-Celsius','Air Temp (?F)','Time (24:00)','Field_Tech','DateVerified','WhoVerified','Year'], axis=1)

In [ ]:
print(data.isna().sum()) #Gives the number of NAN values in every column
data = data.interpolate()

In [ ]:
print(data.isna().sum()) #This shows that all the missing values are interpolated and data is ready to use

In [ ]:
permutation = np.random.permutation(data.shape[0]) #randomly shuffles the rows of the data to reduce bias
data = data.iloc[permutation]

In [ ]:
print(data.head())

In [ ]:
# Visualize missing values as a heatmap (optional for deeper inspection)
sns.heatmap(data.isnull(), cbar=False, cmap="viridis")
plt.title("Missing Data Heatmap")
plt.show()



In [ ]:
# Check for unique values in categorical columns
categorical_columns = ['pH (standard units)', 'Site_Id']

# Print the columns of the dataframe to verify their names
print(data.columns)

for col in categorical_columns:
    # Check if the column exists in the dataframe before trying to access it
    if col in data.columns:
        unique_values = data[col].unique()
        print(f"Unique Values in {col}: {unique_values}")
    else:
        print(f"Column '{col}' not found in the DataFrame.")

In [ ]:
# Test for numerical homogeneity: Identify outliers using the IQR method
numerical_columns = data.select_dtypes(include=['float64', 'int64']).columns
#Filters the dataset to get only numeric columns (e.g., prices, age, scores).
for col in numerical_columns: #Loops through each numeric column to check for outliers.
    Q1 = data[col].quantile(0.25)  # First quartile
    Q3 = data[col].quantile(0.75)  # Third quartile
   # Computes Q1 (25th percentile) and Q3 (75th percentile) for the column.
    IQR = Q3 - Q1  # Interquartile range
   #Calculates IQR (Interquartile Range) — the spread of the middle 50% of the data.
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Sets the boundaries for acceptable values using the IQR method.
    # Anything below lower_bound or above upper_bound is considered an outlier.

    outliers = data[(data[col] < lower_bound) | (data[col] > upper_bound)]
    print(f"Column: {col}\nOutliers Detected: {len(outliers)}")

In [ ]:
# Visualize numerical columns for anomalies
for col in numerical_columns:
    plt.figure(figsize=(8, 4))
    sns.boxplot(x=data[col], color="skyblue")
    plt.title(f"Boxplot for {col}")
    plt.show()

Seaborn automatically calculates Q1, Q3, IQR, and plots:

Box: 25th to 75th percentile

Line inside box: median

Dots outside whiskers: outliers or anomalies

In [ ]:
# Verify consistency of numerical ranges in key columns
print("\nDescriptive Statistics for Key Numerical Columns:")
print(data[numerical_columns].describe())

In [ ]:
#This is the most important step
# Prepare Data for Numerical Correlation

# Convert non-numerical columns to appropriate formats or exclude them from correlation matrix
numerical_data = data.select_dtypes(include=['float64', 'int64'])

# Correlation Matrix to Check Relationships Between Numerical Features
if not numerical_data.empty:
    plt.figure(figsize=(10, 6))

    correlation_matrix = numerical_data.corr()

#  Computes the correlation matrix between all numeric columns.
#  The .corr() method calculates the Pearson correlation coefficient, which ranges from:
#   +1: perfect positive correlation
# 0: no correlation
# -1: perfect negative correlation

    sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", cbar=True)

# Creates a heatmap using Seaborn to visualize the correlation matrix.
# annot=True: Displays the correlation values inside each cell.
# fmt=".2f": Formats those numbers to 2 decimal places.
# cmap="coolwarm": Uses a color gradient from blue (negative) to red (positive).
# cbar=True: Displays a color bar legend on the side.

    plt.title("Correlation Matrix of Numerical Features")
    plt.show()
else:
    print("No numerical data available for correlation analysis.")

In [ ]:
# Verify Distribution of Numerical Columns
for col in numerical_data.columns:
    plt.figure(figsize=(8, 4))
    sns.histplot(data[col], kde=True, color="blue")
    plt.title(f"Distribution of {col}")
    plt.xlabel(col)
    plt.ylabel("Frequency")
    plt.show()

# The **Chi-Square Test** is a statistical test used to determine whether there's a significant association between two categorical variables.

In [ ]:



# Salinity
data['Salinity_Level'] = pd.cut(data['Salinity (ppt)'],
                                 bins=[0, 1, 5, 10],
                                 labels=['Low', 'Medium', 'High'],
                                 include_lowest=True)

# Dissolved Oxygen
data['Oxygen_Level'] = pd.cut(data['Dissolved Oxygen (mg/L)'],
                               bins=[0, 5, 8, 16],
                               labels=['Low', 'Normal', 'High'],
                               include_lowest=True)

# pH
data['pH_Level'] = pd.cut(data['pH (standard units)'],
                           bins=[0, 6.5, 7.5, 10],
                           labels=['Acidic', 'Neutral', 'Alkaline'],
                           include_lowest=True)

# Water Depth
data['Depth_Level'] = pd.cut(data['Water Depth (m)'],
                              bins=[0, 0.5, 1.5, 15],
                              labels=['Shallow', 'Medium', 'Deep'],
                              include_lowest=True)

# Water Temp
data['WaterTemp_Level'] = pd.cut(data['Water Temp (?C)'],
                                  bins=[0, 15, 30, 80],
                                  labels=['Cold', 'Warm', 'Hot'],
                                  include_lowest=True)

# Air Temp
data['AirTemp_Level'] = pd.cut(data['AirTemp (C)'],
                                bins=[-20, 10, 20, 40],
                                labels=['Cold', 'Mild', 'Hot'],
                                include_lowest=True)


In [ ]:
from scipy.stats import chi2_contingency
import pandas as pd
from itertools import combinations

# List of all binned (categorical) columns
binned_cols = [
    'Salinity_Level',
    'Oxygen_Level',
    'pH_Level',
    'Depth_Level',
    'WaterTemp_Level',
    'AirTemp_Level'
]

# Loop through all combinations of two columns
for col1, col2 in combinations(binned_cols, 2):
    print(f"\nChi-Square Test: {col1} vs {col2}")

    contingency = pd.crosstab(data[col1], data[col2])

    if contingency.shape[0] > 1 and contingency.shape[1] > 1:
        chi2, p, dof, expected = chi2_contingency(contingency)
        print("Chi-square statistic:", round(chi2, 3))
        print("p-value:", round(p, 4))

        if p < 0.05:
            print("➡️ Significant association")
        else:
            print("❌ No significant association")
    else:
        print("⚠️ Not enough variation to perform test.")


In [ ]:
# Normality Test for Numerical Columns
from scipy.stats import shapiro

for col in numerical_data.columns:
    stat, p = shapiro(data[col])
    print(f"Shapiro-Wilk Test for {col}:")
    if p > 0.05:
        print(f"P-Value: {p} -> Data appears to be normally distributed.\n")
    else:
        print(f"P-Value: {p} -> Data does not appear to be normally distributed.\n")

In [ ]:
# 1. Display Basic Information About the Dataset
print("\n📋 **Dataset Overview:**\n")
print(data.info())

In [ ]:
# 2. Display the First Few Rows of the Dataset
print("\n🔍 **First 5 Rows of the Dataset:**\n")
print(data.head())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 5. Distribution of Numerical Features
numerical_columns = [
    'Salinity (ppt)',
    'Dissolved Oxygen (mg/L)',
    'pH (standard units)',
    'Water Depth (m)',
    'Water Temp (?C)',
    'AirTemp (C)'
]

for col in numerical_columns:
    plt.figure(figsize=(8, 4))
    sns.histplot(data[col], kde=True, color="skyblue")
    plt.title(f"Distribution of {col}")
    plt.xlabel(col)
    plt.ylabel("Frequency")
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 6. Frequency Distribution of Categorical Features
categorical_columns = [
    'Salinity_Level',
    'Oxygen_Level',
    'pH_Level',
    'Depth_Level',
    'WaterTemp_Level',
    'AirTemp_Level'
]

for col in categorical_columns:
    plt.figure(figsize=(8, 4))
    sns.countplot(x=data[col], palette="coolwarm", order=data[col].value_counts().index)
    plt.title(f"Frequency Distribution of {col}")
    plt.xlabel(col)
    plt.ylabel("Count")
    plt.xticks(rotation=25)
    plt.tight_layout()
    plt.grid(True, axis='y', linestyle='--', alpha=0.5)
    plt.show()


In [ ]:
# 7. Pairplot for Numerical Relationships
sns.pairplot(data[numerical_columns], diag_kind="kde")

# Uses Seaborn's pairplot function to:
# Plot scatterplots for each pair of numerical columns.
# Plot KDE (Kernel Density Estimation) curves on the diagonals instead of histograms (diag_kind="kde").
# data[numerical_columns]: selects only the numerical columns to plot.

plt.suptitle("Pairplot of Numerical Features", y=1.02)

# Adds a super title above the entire grid of plots.
# y=1.02 lifts the title slightly above the plot area to prevent overlap.
plt.show()

In [ ]:
# 8. Boxplots to Check for Outliers in Numerical Features
for col in numerical_columns:
    plt.figure(figsize=(8, 4))
    sns.boxplot(y=data[col], color="lightblue")
    plt.title(f"Boxplot for {col}")
    plt.ylabel(col)
    plt.show()

# Seaborn automatically calculates Q1, Q3, IQR, and plots:
# Box: 25th to 75th percentile
# Line inside box: median
# Dots outside whiskers: outliers or anomalies

In [ ]:
# 9. Column-Wise Analysis
# this give full overview
print("\n🧾 **Column-Wise Analysis:**\n")
for col in data.columns:
    print(f"\n🔹 **{col}**")
    print(f" - Data Type: {data[col].dtype}")
    print(f" - Unique Values: {data[col].nunique()}")
    print(f" - Sample Values: {data[col].unique()[:5]}")
    print(f" - Null Values: {data[col].isnull().sum()}")
    if data[col].dtype in ['int64', 'float64']:
       print(f" - Mean: {data[col].mean():.2f}, Std Dev: {data[col].std():.2f}, Min: {data[col].min()}, Max: {data[col].max()}")

In [ ]:
# 10. Row-Wise Analysis

print("\n🧾 **Row-Wise Analysis:**\n")

# Display a sample of rows with maximum and minimum values for key numerical columns
key_columns = [
    'Salinity (ppt)',
    'Dissolved Oxygen (mg/L)',
    'pH (standard units)',
    'Water Depth (m)',
    'Water Temp (?C)',
    'AirTemp (C)'
]

for col in key_columns:
    print(f"\n🔸 Rows with Maximum and Minimum Values for {col}:")

    max_value = data[col].max()
    min_value = data[col].min()

    max_row = data.loc[data[col] == max_value].iloc[0]
    min_row = data.loc[data[col] == min_value].iloc[0]

    print(f" - Row with MAX {col} (Value: {max_value}):\n{max_row}\n")
    print(f" - Row with MIN {col} (Value: {min_value}):\n{min_row}\n")


In [ ]:
# 11. Duplicate Rows Check (based on specific columns)
print("\n🔍 **Duplicate Rows Check (Based on Key Sensor Columns):**")

# Define the subset of columns to check duplicates
subset_cols = [
    'Salinity (ppt)',
    'Dissolved Oxygen (mg/L)',
    'pH (standard units)',
    'Water Depth (m)',
    'Water Temp (?C)',
    'AirTemp (C)'
]

# Count duplicates based on those columns
duplicate_count = data.duplicated(subset=subset_cols).sum()
print(f" - Total Duplicate Rows (based on sensor readings): {duplicate_count}")

# If duplicates exist, display them
if duplicate_count > 0:
    print(" - Displaying Duplicate Rows:\n")
    print(data[data.duplicated(subset=subset_cols, keep=False)])


In [ ]:
# 13. Row Integrity: Check for Rows with Extreme Low/High Values in Numerical Columns
print("\n📊 **Row Integrity Check for Extreme Values:**")
thresh_low = 0.05  # 5% threshold for low values
thresh_high = 0.95  # 95% threshold for high values

for col in key_columns:
    low_value_threshold = data[col].quantile(thresh_low)
    high_value_threshold = data[col].quantile(thresh_high)
    print(f"\nColumn: {col}")
    print(f" - Rows below {thresh_low*100}% threshold ({low_value_threshold}): {len(data[data[col] < low_value_threshold])}")
    print(f" - Rows above {thresh_high*100}% threshold ({high_value_threshold}): {len(data[data[col] > high_value_threshold])}")

In [ ]:
print("\n📏 **Logical Range Check for Numerical Columns:**")

# Define acceptable ranges for each numerical column
valid_ranges = {
    'Salinity (ppt)': (0, 50),  # Salinity shouldn't be negative; 0–50 is safe
    'Dissolved Oxygen (mg/L)': (0, 20),  # DO is always ≥ 0, rarely > 20
    'pH (standard units)': (0, 14),  # pH scale is 0 to 14
    'Water Depth (m)': (0, 50),  # Depth can't be negative; max depends on dataset
    'Water Temp (?C)': (0, 100),  # Water boils at 100°C
    'AirTemp (C)': (-50, 60)  # Extreme air temps range from -50 to 60°C
}

# Check each column for out-of-range values
for col, (min_val, max_val) in valid_ranges.items():
    outliers = data[(data[col] < min_val) | (data[col] > max_val)]

    print(f"\n🔎 Checking {col}:")
    if outliers.empty:
        print(f" - All values are within the valid range ({min_val} to {max_val}).")
    else:
        print(f" - Found {len(outliers)} out-of-range value(s)!")
        print(outliers[[col]])  # print only the column with the problem


In [ ]:
from itertools import combinations

print("\n🗂️ **Cross-Validation of Categorical Columns:**")

categorical_columns = [
    'Salinity_Level',
    'Oxygen_Level',
    'pH_Level',
    'Depth_Level',
    'WaterTemp_Level',
    'AirTemp_Level'
]

# Loop through all unique pairs of categorical columns
for col1, col2 in combinations(categorical_columns, 2):
    print(f"\n🔄 Unique combinations of {col1} and {col2}:")
    combo_counts = data.groupby([col1, col2]).size().reset_index(name='Count')
    print(combo_counts)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 26. Pairplot for Numerical Relationships
sns.pairplot(
    data,
    hue='Site_Id',
    vars=[
        'Salinity (ppt)',
        'Dissolved Oxygen (mg/L)',
        'pH (standard units)',
        'Water Depth (m)',
        'Water Temp (?C)',
        'AirTemp (C)'
    ],
    palette="husl"
)

plt.suptitle("Pairplot of Key Numerical Features by Site_Id", y=1.02)
plt.show()


# 26. Pairplot for Numerical Relationships
sns.pairplot(
    data,
    hue='Site_Id',
    vars=[
        'Salinity (ppt)',
        'Dissolved Oxygen (mg/L)',
        'pH (standard units)',
        'Water Depth (m)',
        'Water Temp (?C)',
        'AirTemp (C)'
    ],
    palette="husl"
)

plt.suptitle("Pairplot of Key Numerical Features by Site_Id", y=1.02)
plt.show()


In [ ]:


# 28. pH Distribution by Depth Level (Box Plot)
plt.figure(figsize=(10, 6))
sns.boxplot(x='Depth_Level', y='pH (standard units)', data=data, palette="coolwarm")
plt.title("pH Distribution by Depth Level")
plt.xlabel("Depth Level")
plt.ylabel("pH (standard units)")
plt.grid(True, axis='y', linestyle='--', alpha=0.6)
plt.show()


In [ ]:


# 33. Dissolved Oxygen vs pH (Hexbin Plot)
plt.figure(figsize=(10, 6))
plt.hexbin(
    data['Dissolved Oxygen (mg/L)'],
    data['pH (standard units)'],
    gridsize=20,
    cmap='Blues'
)
plt.colorbar(label='Count')
plt.title("Dissolved Oxygen vs pH (Hexbin Plot)")
plt.xlabel("Dissolved Oxygen (mg/L)")
plt.ylabel("pH (standard units)")
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Convert Site_Id to string type first, then to a categorical type for plotting
data['Site_Id_Categorical'] = data['Site_Id'].astype(str).astype('category')

# 35. DO vs Site_Id, colored by actual pH value
plt.figure(figsize=(12, 6))
scatter = plt.scatter(
    data['Site_Id_Categorical'], # Use the categorical version for plotting
    data['Dissolved Oxygen (mg/L)'],
    c=data['pH (standard units)'],
    cmap='coolwarm',
    edgecolors='black',
    s=70
)

plt.colorbar(scatter, label='pH (standard units)')
plt.title("Dissolved Oxygen vs Site_Id (colored by pH value)")
plt.xlabel("Site_Id")
plt.ylabel("Dissolved Oxygen (mg/L)")
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
data.head()